In [1]:
import os
import numpy as np

from tqdm.notebook import tqdm
from sklearn.metrics import auc

from tensorflow.keras.datasets.fashion_mnist import load_data
from image_similarity_measures.quality_metrics import rmse

In [2]:
(_, _), (x_test, y_test) = load_data()

In [3]:
x_test = x_test.reshape(-1, 28, 28, 1)

In [4]:
x_test = x_test.astype('float32') / 255.

In [10]:
prioritizations = []

for i in tqdm(range(len(x_test))):
    prioritizations.append(np.argsort([rmse(x_test[i], x_test[j]) for j in range(len(x_test))]))

KeyboardInterrupt: 

In [12]:
np.save('./tmp/sim_prioritizations.npy', prioritizations)

In [5]:
prioritizations = np.load('./tmp/sim_prioritizations.npy')

In [6]:
y_pred = np.load('./tmp/fashion_mnist_test_pred.npy')

In [7]:
# calculate auc scores for all similarity-based prioritizations

aucs_prioritized = []

for i in tqdm(range(len(prioritizations))):
    prioritization = prioritizations[i]
    
    y_test_sorted = y_test[prioritization]
    y_pred_sorted = y_pred[prioritization]
    
    errors = 0
    cum_errors = []

    for i in range(len(y_test_sorted)):
        if y_test_sorted[i] != y_pred_sorted[i]:
            errors += 1
        cum_errors.append(errors)
        
    auc_prioritized = auc(range(len(y_test_sorted)), cum_errors)
    aucs_prioritized.append(auc_prioritized)

In [8]:
# select the best and the worst similarity-based prioritizations

print(np.argmax(aucs_prioritized))
print(np.argmin(aucs_prioritized))

681
2761


In [9]:
# save the best similarity-based prioritization
prioritization = np.argsort([rmse(x_test[np.argmax(aucs_prioritized)], x_test[j]) for j in range(len(x_test))])

np.save('./tmp/sim.npy', prioritization)